In [117]:
# import necessary libraries
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

print('Libraries imported!')

Libraries imported!


In [118]:
# get the us city data 
URL = 'https://raw.githubusercontent.com/camellre/Coursera_Capstone/master/uscitiesv1.5.csv'
data = pd.read_csv(URL)

In [119]:
# explorer the dataset
data.head()

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,population_proper,density,source,incorporated,timezone,zips,id
0,Prairie Ridge,Prairie Ridge,WA,Washington,53053,Pierce,47.1443,-122.1408,NaN,NaN,1349.8,polygon,False,America/Los_Angeles,98360 98391,1840037882
1,Edison,Edison,WA,Washington,53057,Skagit,48.5602,-122.4311,NaN,NaN,127.4,polygon,False,America/Los_Angeles,98232,1840017314
2,Packwood,Packwood,WA,Washington,53041,Lewis,46.6085,-121.6702,NaN,NaN,213.9,polygon,False,America/Los_Angeles,98361,1840025265
3,Wautauga Beach,Wautauga Beach,WA,Washington,53035,Kitsap,47.5862,-122.5482,NaN,NaN,261.7,point,False,America/Los_Angeles,98366,1840037725
4,Harper,Harper,WA,Washington,53035,Kitsap,47.5207,-122.5196,NaN,NaN,342.1,point,False,America/Los_Angeles,98366,1840037659


In [120]:
# extract the California cities with densities that are higher than 1000.0
cities = data.loc[:,['city_ascii', 'state_id', 'lat', 'lng', 'density']].copy()
cities = cities.rename(index=str, columns={'city_ascii':'city','state_id':'state'})
cities_t = cities[cities['state'] == 'CA'].reset_index(drop=True)
CA_cities = cities_t[cities_t['density']>=1000.0].reset_index(drop=True).copy()

# explorer the dataset
print(CA_cities.shape[:])
CA_cities.head()

(605, 5)


,city,state,lat,lng,density
0,Kensington,CA,37.9084,-122.2805,1849.8
1,Pleasant Hill,CA,37.9540,-122.0759,1909.0
2,Pomona,CA,34.0585,-117.7625,2572.0
3,Oakdale,CA,37.7616,-120.8470,1446.0
4,Knights Landing,CA,38.7979,-121.7176,1199.0


In [121]:
# define the foursquare API credentials
CLIENT_ID = '13XR3N4JCCURJANAMONLKKUZBBVO4LTDTBPDU2XNA1N5ANKX'
CLIENT_SECRET = 'CGKLUUMIQKZXEXDH2XDZINBX4PQNOOHWMIQCCU5BRVRJPL2V'
VERSION = '20190401'
limit = 1000

In [122]:
# define a function that return a dataset that contains venues for each city
def getNearbyVenues(cities, latitudes, longitudes):
    
    venues_list=[]
    for city, lat, lng in zip(cities, latitudes, longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            limit)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            city,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        print(city)

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['city',
                  'city Latitude', 
                  'city Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [123]:
# exectue the function and create the dataset
cities_venues = getNearbyVenues(cities = CA_cities['city'], latitudes = CA_cities['lat'], longitudes= CA_cities['lng'])

Kensington
Pleasant Hill
Pomona
Oakdale
Knights Landing
Upland
Riverbank
Montrose
Brea
Exeter
Hercules
Moorpark
Windsor
El Monte
Sebastopol
Burbank
Discovery Bay
Rancho Santa Margarita
Komandorski Village
Casa Conejo
South Whittier
Manhattan Beach
Orangevale
Quartz Hill
Maywood
Thousand Palms
East Palo Alto
South San Gabriel
Lakewood
Irvine
Windsor Hills
Sunset Beach
Sunnymead
Rocklin
Salinas
Poplar
San Miguel
Newark
Del Rosa
Bell Gardens
Lodi
Foster City
Orcutt
Fairview
Vallejo
La Riviera
Indio
Sunnyside
Azusa
Narod
Emeryville
Hollister
Fair Oaks
Meiners Oaks
Valinda
Winter Gardens
Buellton
Waterford
McKinleyville
Weedpatch
Cayucos
Fallbrook
Emerald Bay
Buena Park
Panorama Heights
Shore Acres
Saranap
Kelseyville
Downey
Castle Park
West Athens
Santa Ana Heights
El Segundo
Los Altos
Cutler
San Bernardino
Encinitas
Rubidoux
American Canyon
Whittier
Pacheco
Duarte
Sanger
Chula Vista
Rio Del Mar
Sutter
Cupertino
Terra Bella
Stratford
Charter Oak
Edgemont
Ukiah
Fullerton
Lompoc
East Richmon

In [132]:
# save the dataset into file in case of unnecessary and more API calling
cities_venues.to_csv('CA_cities', encoding='utf-8', index=False)

In [134]:
# explorer the dataset
print(cities_venues.shape[:])
cities_venues.head()

(52166, 7)


,city,city Latitude,city Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Kensington,37.9084,-122.2805,The Little Farm,37.909633,-122.264792,Farm
1,Kensington,37.9084,-122.2805,Blake Garden,37.912217,-122.281758,Garden
2,Kensington,37.9084,-122.2805,Indian Rock Park,37.892207,-122.273088,Park
3,Kensington,37.9084,-122.2805,Zachary's Chicago Pizza,37.891453,-122.278608,Pizza Place
4,Kensington,37.9084,-122.2805,Rivoli,37.891095,-122.286327,New American Restaurant


In [135]:
# explorer how many venues for each city
cities_venues.groupby('city').count()

,city Latitude,city Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
city,,,,,,
Agoura Hills,100,100,100,100,100,100
Alameda,100,100,100,100,100,100
Albany,100,100,100,100,100,100
Alhambra,100,100,100,100,100,100
Aliso Viejo,100,100,100,100,100,100
Alondra Park,100,100,100,100,100,100
Alpaugh,37,37,37,37,37,37
Alpine,68,68,68,68,68,68
Alta Loma,100,100,100,100,100,100


In [136]:
# get the number of unique categories 
print('There are {} uniques categories.'.format(len(cities_venues['Venue Category'].unique())))

There are 488 uniques categories.


In [139]:
# one hot encoding
CA_onehot = pd.get_dummies(cities_venues[['Venue Category']], prefix="", prefix_sep="")

CA_onehot['city'] = cities_venues['city'] 

fixed_columns = [CA_onehot.columns[-1]] + list(CA_onehot.columns[:-1])
CA_onehot = CA_onehot[fixed_columns].copy()

# explorer the new dataset for a more clear category view
print(CA_onehot.shape[:])
CA_onehot.head()

(52166, 489)


,city,ATM,Accessories Store,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Apres Ski Bar,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Bay,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Bowling Green,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Canal,Candy Store,Cantonese Restaurant,Car Wash,Caribbean Restaurant,Casino,Chaat Place,Cheese Shop,Child Care Service,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Circus,City,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Baseball Diamond,College Basketball Court,College Bookstore,College Cafeteria,College Football Field,College Gym,College Library,College Rec Center,College Soccer Field,College Stadium,College Track,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Community College,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,County,Courthouse,Credit Union,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Donburi Restaurant,Donut Shop,Drive-in Theater,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Fabric Shop,Factory,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Hunan Restaurant,Hunting Supply,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Inn,Insurance Office,Internet Cafe,Intersection,Irish Pub,Island,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Lebanese Restaurant,Library,Light Rail Station,Lighthouse,Lighting Store,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Middle School,M

In [142]:
# calculate the mean of categories for each city
CA_grouped = CA_onehot.groupby('city').mean().reset_index()

# explorer the new dataset
print(CA_grouped.shape)
CA_grouped.head()

(605, 489)


,city,ATM,Accessories Store,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Apres Ski Bar,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Bay,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Bowling Green,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Canal,Candy Store,Cantonese Restaurant,Car Wash,Caribbean Restaurant,Casino,Chaat Place,Cheese Shop,Child Care Service,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Circus,City,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Baseball Diamond,College Basketball Court,College Bookstore,College Cafeteria,College Football Field,College Gym,College Library,College Rec Center,College Soccer Field,College Stadium,College Track,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Community College,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,County,Courthouse,Credit Union,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Donburi Restaurant,Donut Shop,Drive-in Theater,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Fabric Shop,Factory,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Hunan Restaurant,Hunting Supply,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Inn,Insurance Office,Internet Cafe,Intersection,Irish Pub,Island,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Lebanese Restaurant,Library,Light Rail Station,Lighthouse,Lighting Store,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Middle School,M

In [144]:
# define a function that returns the most common venues for each city
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [146]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['city']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
cities_venues_sorted = pd.DataFrame(columns=columns)
cities_venues_sorted['city'] = CA_grouped['city']

for ind in np.arange(CA_grouped.shape[0]):
    cities_venues_sorted.iloc[ind, 1:] = return_most_common_venues(CA_grouped.iloc[ind, :], num_top_venues)

# explorer the sorted dataset that will be used for modeling
print(cities_venues_sorted.shape)
cities_venues_sorted.head()

(605, 11)


,city,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agoura Hills,Deli / Bodega,Breakfast Spot,Brewery,American Restaurant,Hotel,Gym / Fitness Center,Mexican Restaurant,Fast Food Restaurant,Park,Wine Bar
1,Alameda,Café,Beach,Coffee Shop,Mexican Restaurant,Grocery Store,Deli / Bodega,Sushi Restaurant,Wine Shop,Gym,Burger Joint
2,Albany,Coffee Shop,Pizza Place,Grocery Store,Trail,Flower Shop,Brewery,Mexican Restaurant,Bar,Breakfast Spot,Indian Restaurant
3,Alhambra,Chinese Restaurant,Mexican Restaurant,Convenience Store,Burger Joint,Italian Restaurant,Bakery,Dessert Shop,Park,Vietnamese Restaurant,Szechuan Restaurant
4,Aliso Viejo,Park,Pizza Place,Grocery Store,Burger Joint,Bakery,Sushi Restaurant,Breakfast Spot,Mexican Restaurant,Thai Restaurant,American Restaurant
5,Alondra Park,Japanese Restaurant,Burger Joint,Convenience Store,Cosmetics Shop,Noodle House,Coffee Shop,Mediterranean Restaurant,Vietnamese Restaurant,Pizza Place,Fast Food Restaurant
6,Alpaugh,Fast Food Restaurant,Sandwich Place,Mexican Restaurant,Discount Store,Pizza Place,Convenience Store,Pharmacy,Dessert Shop,Bank,Mobile Phone Shop
7,Alpine,Casino,Mexican Restaurant,Clothing Store,American Restaurant,Shoe Store,Women's Store,Café,Accessories Store,Italian Restaurant,Bar
8,Alta Loma,Mexican Restaurant,Coffee Shop,Burger Joint,Sushi Restaurant,Gym / Fitness Center,Grocery Store,Pizza Place,Sandwich Place,Asian Restaurant,Fast Food Restaurant
9,Altadena,Trail,Scenic Lookout,Pizza Place,Coffee Shop,Café,Fast Food Restaurant,Grocery Store,American Restaurant,Mexican Restaurant,Burger Joint
